In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
pd.set_option('display.max_columns', None) # displays all columns in dfs
#%matplotlib inline
import warnings
warnings.filterwarnings('ignore') # no more warnings
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error as mae

In [2]:
fifa = pd.read_csv("fifa21_male2.csv")
fifa

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [3]:
fifa.columns

Index(['ID', 'Name', 'Age', 'OVA', 'Nationality', 'Club', 'BOV', 'BP',
       'Position', 'Player Photo',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'Gender'],
      dtype='object', length=107)

In [4]:
fifa.columns = [fifa.columns[i].lower().replace(' ','_') for i in range (len(fifa.columns))]
fifa.columns

Index(['id', 'name', 'age', 'ova', 'nationality', 'club', 'bov', 'bp',
       'position', 'player_photo',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'gender'],
      dtype='object', length=107)

In [5]:
fifa = fifa.rename(columns={'ova': 'overall', 'bov': 'best_overall','ls': 'left_striker','st':'left_striker',
                           'rs':'right_striker','lw':'left_wing','lf':'left_forward','cf':'central_forward','rf':'right_forward',
                            'rw':'right_wing','lam':'left_attacking_midfielder','cam':'central_attacking_midfileder','ram':'right_attacking_midfielder',
                           'lm':'left_midfielder','lcm':'left_central_midfielder','cm':'central_midfielder','rcm':'right_central_midfielder','rm':'right_midfielder',
                           'lwb':'left_wing_back','ldm':'left_deffensive_midfielder','cdm':'central_defensive_midfielder','rdm':'right_defensive_midfielder',
                           'rwb':'right_wing_back','lb':'left_back','lcb':'left_central_back','cb':'central_back','rcb':'right_central_back','rb':'right_back',
                           'gk':'goal_keeper','bp':'best_position','fk_accuracy':'free_kick_accuracy','a/w':'attacking_work_rate','d/w':'defensive_work_rate','sm':'skill_moves','w/f':'week_foot_ability','sho':'shooting',
                           'pot':'potential'})
fifa

,id,name,age,overall,nationality,club,best_overall,best_position,position,player_photo,club_logo,flag_photo,potential,team_&_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,free_kick_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,week_foot_ability,skill_moves,attacking_work_rate,defensive_work_rate,ir,pac,shooting,pas,dri,def,phy,hits,left_striker,left_striker,right_striker,left_wing,left_forward,central_forward,right_forward,right_wing,left_attacking_midfielder,central_attacking_midfileder,right_attacking_midfielder,left_midfielder,left_central_midfielder,central_midfielder,right_central_midfielder,right_midfielder,left_wing_back,left_deffensive_midfielder,central_defensive_midfielder,right_defensive_midfielder,right_wing_back,left_back,left_central_back,central_back,right_central_back,right_back,goal_keeper,gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€1

In [6]:
fifa_new = fifa[['name', 'age', 'overall', 'nationality', 'best_position','potential','team_&_contract',
                                   'value','wage','ir','gender']].copy()
fifa_new

,name,age,overall,nationality,best_position,potential,team_&_contract,value,wage,ir,gender
0,G. Pasquale,33,69,Italy,LWB,69,Udinese 2008 ~ 2016,€625K,€7K,2 ★,Male
1,Luis García,37,71,Spain,CM,71,KAS Eupen 2014 ~ 2019,€600K,€7K,1 ★,Male
2,J. Cole,33,71,England,CAM,71,Coventry City 2016 ~ 2020,€1.1M,€15K,2 ★,Male
3,D. Yorke,36,68,Trinidad &amp; Tobago,ST,82,Sunderland 2009,€0,€0,1 ★,Male
4,Iniesta,36,81,Spain,CAM,81,Vissel Kobe 2018 ~ 2021,€5.5M,€12K,4 ★,Male
...,...,...,...,...,...,...,...,...,...,...,...
17120,A. Medioub,22,64,France,CB,69,"CD Tondela Jun 30, 2021 On Loan",€550K,€2K,1 ★,Male
17121,W. Rickard,18,56,Wales,CAM,66,Burnley 2020 ~ 2021,€130K,€3K,1 ★,Male
17122,C. Barrett,18,56,England,RB,67,Burnley 2020 ~ 2022,€130K,€2K,1 ★,Male
17123,J. Gazibegović,20,62,Bosnia Herzegovina,RB,73,SK Sturm Graz 2020 ~ 2023,€475K,€3K,1 ★,Male


In [7]:
fifa_new.isnull().sum()

name               0
age                0
overall            0
nationality        0
best_position      0
potential          0
team_&_contract    0
value              0
wage               0
ir                 0
gender             0
dtype: int64

In [8]:
fifa_new.best_position.unique()

array(['LWB', 'CM', 'CAM', 'ST', 'RW', 'CB', 'CF', 'GK', 'CDM', 'RWB',
       'LB', 'LM', 'RM', 'RB', 'LW'], dtype=object)

In [9]:
fifa_gk = fifa_new[(fifa_new["best_position"] == "GK")]
fifa_gk

,name,age,overall,nationality,best_position,potential,team_&_contract,value,wage,ir,gender
14,O. Kahn,38,87,Germany,GK,90,FC Bayern München 2008,€0,€0,4 ★,Male
18,M. Schwarzer,42,73,Australia,GK,73,Leicester City 2015 ~ 2016,€500K,€15K,2 ★,Male
19,J. Lehmann,39,79,Germany,GK,87,VfB Stuttgart 2010,€0,€0,4 ★,Male
20,G. Király,39,69,Hungary,GK,69,Hungary Free,€0,€0,2 ★,Male
26,G. Buffon,42,82,Italy,GK,82,Juventus 2019 ~ 2021,€2.2M,€41K,4 ★,Male
...,...,...,...,...,...,...,...,...,...,...,...
17039,T. Seymour,18,51,England,GK,67,Portsmouth 2020 ~ 2021,€60K,€500,1 ★,Male
17061,G. Crespi,19,49,Italy,GK,65,Crotone 2020 ~ 2024,€50K,€550,1 ★,Male
17091,B. Reus,19,58,Netherlands,GK,69,AZ Alkmaar 2020 ~ 2021,€170K,€550,1 ★,Male
17100,J. Stejskal,23,70,Czech Republic,GK,78,SK Slavia Praha 2020 ~ 2020,€2M,€500,1 ★,Male


In [10]:
fifa_defense = fifa_new[(fifa_new["best_position"] == 'LWB') | (fifa_new["best_position"] == 'RWB')|(fifa_new["best_position"] == 'CB')|
                    (fifa_new["best_position"] == 'LB')|(fifa_new["best_position"] == 'RB')]
fifa_defense

,name,age,overall,nationality,best_position,potential,team_&_contract,value,wage,ir,gender
0,G. Pasquale,33,69,Italy,LWB,69,Udinese 2008 ~ 2016,€625K,€7K,2 ★,Male
8,G. Neville B,35,76,England,CB,82,Manchester United 1991 ~ 2011,€0,€0,3 ★,Male
10,P. Neville,35,74,England,CB,74,Everton 2005 ~ 2013,€120K,€7K,2 ★,Male
13,S. Campbell,35,75,England,CB,79,Newcastle United 2010 ~ 2011,€0,€0,3 ★,Male
15,D. Hamann,36,61,Germany,CB,82,Milton Keynes Dons 2010 ~ 2011,€0,€0,3 ★,Male
...,...,...,...,...,...,...,...,...,...,...,...
17113,Yan Couto,18,64,Brazil,RWB,80,Girona FC 2020 ~ 2025,€850K,€650,1 ★,Male
17116,J. Lawrence,17,52,Germany,CB,73,Bayern München II 2020 ~ 2022,€100K,€500,1 ★,Male
17120,A. Medioub,22,64,France,CB,69,"CD Tondela Jun 30, 2021 On Loan",€550K,€2K,1 ★,Male
17122,C. Barrett,18,56,England,RB,67,Burnley 2020 ~ 2022,€130K,€2K,1 ★,Male


In [11]:
fifa_midfielders = fifa_new[(fifa["best_position"] == 'CM') | (fifa_new["best_position"] == 'CAM')|(fifa_new["best_position"] == 'CDM')|
                    (fifa_new["best_position"] == 'LM')|(fifa_new["best_position"] == 'RM')]
fifa_midfielders

,name,age,overall,nationality,best_position,potential,team_&_contract,value,wage,ir,gender
1,Luis García,37,71,Spain,CM,71,KAS Eupen 2014 ~ 2019,€600K,€7K,1 ★,Male
2,J. Cole,33,71,England,CAM,71,Coventry City 2016 ~ 2020,€1.1M,€15K,2 ★,Male
4,Iniesta,36,81,Spain,CAM,81,Vissel Kobe 2018 ~ 2021,€5.5M,€12K,4 ★,Male
6,E. Belözoğlu,38,77,Turkey,CM,77,Fenerbahçe SK 2019 ~ 2020,€2.8M,€44K,2 ★,Male
7,R. Giggs,39,78,Wales,CAM,78,Manchester United 1990 ~ 2014,€1K,€60K,3 ★,Male
...,...,...,...,...,...,...,...,...,...,...,...
17110,Pau Víctor,18,63,Spain,CAM,76,Girona FC 2019 ~ 2021,€650K,€800,1 ★,Male
17114,S. Eide,18,52,Norway,CAM,67,SK Brann 2020 ~ 2024,€100K,€500,1 ★,Male
17115,K. Torgersen,17,51,Norway,CAM,70,Stabæk Fotball 2020 ~ 2023,€70K,€500,1 ★,Male
17117,S. Diarrassouba,18,58,Burkina Faso,RM,77,FC St. Gallen 2020 ~ 2021,€250K,€900,1 ★,Male


In [12]:
fifa_attackers = fifa_new[(fifa_new["best_position"] == 'ST') | (fifa_new["best_position"] == 'RW')|(fifa_new["best_position"] == 'CF')|
                    (fifa_new["best_position"] == 'LW')]
fifa_attackers

,name,age,overall,nationality,best_position,potential,team_&_contract,value,wage,ir,gender
3,D. Yorke,36,68,Trinidad &amp; Tobago,ST,82,Sunderland 2009,€0,€0,1 ★,Male
5,D. Odonkor,27,66,Germany,RW,70,Alemannia Aachen 2011 ~ 2012,€725K,€5K,2 ★,Male
12,R. Keane,35,80,Republic of Ireland,CF,80,LA Galaxy 2011 ~ 2022,€5.5M,€10K,2 ★,Male
16,A. Okocha,33,73,Nigeria,CF,90,Hull City 2009,€0,€0,2 ★,Male
17,Zulu,26,66,Brazil,ST,72,Pohang Steelers 2005 ~ 2013,€0,€0,1 ★,Male
...,...,...,...,...,...,...,...,...,...,...,...
17095,L. Delap,17,61,England,ST,82,Manchester City 2020 ~ 2023,€550K,€3K,1 ★,Male
17104,Tiago Morais,16,61,Portugal,ST,82,Boavista FC 2020 ~ 2024,€550K,€500,1 ★,Male
17111,Adrián Turmo,19,62,Spain,ST,76,Girona FC 2020 ~ 2021,€650K,€1K,1 ★,Male
17118,R. Shanley,19,54,Scotland,ST,68,Hibernian 2018 ~ 2022,€150K,€900,1 ★,Male


In [13]:
fifa_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17125 entries, 0 to 17124
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             17125 non-null  object
 1   age              17125 non-null  int64 
 2   overall          17125 non-null  int64 
 3   nationality      17125 non-null  object
 4   best_position    17125 non-null  object
 5   potential        17125 non-null  int64 
 6   team_&_contract  17125 non-null  object
 7   value            17125 non-null  object
 8   wage             17125 non-null  object
 9   ir               17125 non-null  object
 10  gender           17125 non-null  object
dtypes: int64(3), object(8)
memory usage: 1.4+ MB


In [14]:
fifa_new["value"].replace({"K":"*1e3", "M":"*1e6"}, regex=True).map(pd.eval).astype(int)
fifa_new

SyntaxError: invalid character in identifier (<unknown>, line 1)

In [ ]:
fifa_new["wage"].replace({"K":"*1e3", "M":"*1e6"}, regex=True).map(pd.eval).astype(int)

In [ ]:
sns.pairplot(fifa_attackers)

In [ ]:
rslt_df = fifa_attackers[fifa_attackers['overall'] > 85] 
len(rslt_df)